In [1]:
# import necessay modules
import torch
import torchinfo
import torchvision
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
from torch.utils.data import Dataset, DataLoader # Gives easier dataset managment by creating mini batches etc.
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from tqdm import tqdm # for nice 

In [2]:
# device config
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu') # 

In [3]:
# Hyperparameter
in_channel = 3 # for CIFAR10 32*32
num_classes = 10 # 0-9
learning_rate = 0.001
batch_size = 64
num_epochs = 3

In [4]:
# Load Data
# shape=3, 32, 32
train_dataset = datasets.CIFAR10(
    root="./data", train=True, transform=transforms.ToTensor(), download=True
)
test_dataset = datasets.CIFAR10(
    root="./data", train=False, transform=transforms.ToTensor(), download=True
)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
for batch_idx, (data, targets) in enumerate((train_loader)):
    print(data.shape)
    if(batch_idx==2): break

torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])


In [6]:
# create CNN layer:
class CNNNet(nn.Module):
    def __init__(self, in_channels = 3, num_classes=10):
        super(CNNNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels= 6, kernel_size=(5,5)) # output = (32-5+2*0)/1+1=28 -> [1, 6,28,28] parameter: 5^2*3*6+6 = 456 and colROw: 456*28*28
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2)) # output= 28-2/2+1=13+1= 14 => [1,6,14,14] params =null col_Row=null
        self.conv2 = nn.Conv2d(in_channels = 6, out_channels= 16, kernel_size=(5,5)) # output = 14-5/1 + 1=10=>[1,16,10,10] parameter = 5^2*6*16+16 = 2416 and colRow = 2416*10*10
        self.conv3 = nn.Conv2d(16,120,5)
        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(120,64)
        self.fc2 = nn.Linear(64,num_classes)
    
    def forward(self,x):
        x = F.relu(self.conv1(x)) # output = (32-5+2*0)/1+1=28 -> [1, 6,28,28] parameter: 5^2*3*6+6 = 456 and colROw: 456*28*28
        x = self.pool(x)  # output= 28-2/2+1=13+1= 14 => [1,6,14,14] params =null col_Row=null
        x = F.relu(self.conv2(x)) # output = 14-5/1 + 1=10=>[1,16,10,10] parameter = 5^2*6*16+16 = 2416 and colRow = 2416*10*10
        x = self.pool(x) # output= 10-2/2+1= 4+1= 5 => [1,16,5,5] params =null col_Row=null
        x = F.relu(self.conv3(x)) # output = 5-5/1 + 1= 1 =>[1,120,1,1]; parameter = 5^2*16*120+120 = 48120 and colRow = 48120*1*1
        # x = x.reshape(x.shape[0], -1)
        x = self.flat(x) # output= [1, 120*1*1]=>[1, 120]; params = null;  colRow =null
        x = F.relu(self.fc1(x)) # output:[1, 64]; params: 120*64+64= 7744; colRow= 7744*1=7744
        x = self.fc2(x) # output:[1, 10]; params: 64*10+10= 650; colRow =650
        return x
model = CNNNet().to(device)
torchinfo.summary(model, input_size=(1, 3, 32, 32), col_names= ("input_size", "output_size", "num_params", "mult_adds"))

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Mult-Adds
CNNNet                                   [1, 3, 32, 32]            [1, 10]                   --                        --
├─Conv2d: 1-1                            [1, 3, 32, 32]            [1, 6, 28, 28]            456                       357,504
├─MaxPool2d: 1-2                         [1, 6, 28, 28]            [1, 6, 14, 14]            --                        --
├─Conv2d: 1-3                            [1, 6, 14, 14]            [1, 16, 10, 10]           2,416                     241,600
├─MaxPool2d: 1-4                         [1, 16, 10, 10]           [1, 16, 5, 5]             --                        --
├─Conv2d: 1-5                            [1, 16, 5, 5]             [1, 120, 1, 1]            48,120                    48,120
├─Flatten: 1-6                           [1, 120, 1, 1]            [1, 120]                  --                        --
├─L

In [7]:
model = torchvision.models.vgg16(pretrained= True)
print(model)

c:\Users\libd7\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\libd7\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [8]:
## change the model of the last layer
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
    def forward(self, x):
        return x

In [9]:
## Fine tuning
def fine_tuning(model):
    for param in model.parameters():
        param.requires_grad = False

In [10]:
fine_tuning(model)
model.avgpool = Identity()
model.classifier = nn.Sequential(
    nn.Linear(512,100),
    nn.ReLU(),
    nn.Linear(100,10)
)
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [11]:
# Loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [12]:
# Train Network
def train(num_epochs, train_loader, model, device, loss_fn, optimizer):
    for epoch in range(num_epochs):
        losses=[] 
        for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
            # Get data to cuda if possible
            data = data.to(device=device)
            targets = targets.to(device=device)

            # Forward
            scores = model(data)
            loss = loss_fn(scores, targets)
            losses.append(loss.item())

            # Backward
            optimizer.zero_grad()
            loss.backward()

            # Gradient descent or adam step
            optimizer.step()
        mean_loss=sum(losses)/len(losses)
        print(f"at {epoch} epoch loss: {mean_loss:.5f}")

In [13]:
train(num_epochs, train_loader, model, device, loss_fn, optimizer)

  0%|          | 0/782 [00:00<?, ?it/s]

100%|██████████| 782/782 [03:48<00:00,  3.43it/s]


at 0 epoch loss: 1.24810


100%|██████████| 782/782 [03:55<00:00,  3.31it/s]


at 1 epoch loss: 1.09570


100%|██████████| 782/782 [03:55<00:00,  3.32it/s]

at 2 epoch loss: 1.04556


In [14]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    # We don't need to keep track of gradients here so we wrap it in torch.no_grad()
    with torch.no_grad():
        # Loop through the data
        for x, y in loader:

            # Move data to device
            x = x.to(device=device)
            y = y.to(device=device)

            # Forward pass
            scores = model(x)
            _, predictions = scores.max(1)

            # Check how many we got correct
            num_correct += (predictions == y).sum()

            # Keep track of number of samples
            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples

In [15]:
# Check accuracy on training & test to see how good our model
print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

Accuracy on training set: 65.22
Accuracy on test set: 62.22
